In [1]:
import numpy as np
from ogb.lsc import PCQM4Mv2Dataset
import preprocessing.graph_preprocessing as gp
import pickle
import os

In [2]:
dataset = PCQM4Mv2Dataset(root = "/home/michpir/Documents/PROJECTS", only_smiles = True)

In [4]:
split_dict = dataset.get_idx_split()
train_idx = split_dict['train'] # numpy array storing indices of training molecules
valid_idx = split_dict['valid'] # numpy array storing indices of validation molecules
testdev_idx = split_dict['test-dev'] # numpy array storing indices of test-dev molecules
testchallenge_idx = split_dict['test-challenge'] # numpy array storing indices of test-challenge molecules

In [7]:
train_dataset = []
for index in train_idx:
    train_dataset.append(dataset[index])


In [ ]:
data_list_train = gp.create_graph_data(train_dataset, np.Inf, np.Inf, np.Inf, input_source="homo-lumo")

/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/preprocessing/preprocessing_utils.py:41: UserWarning: Number of group is greater than one hot dimension representation
  warnings.warn("Number of group is greater than one hot dimension representation")
[23:56:40] WARNING: not removing hydrogen atom without neighbors
[23:56:40] WARNING: not removing hydrogen atom without neighbors
[23:56:40] WARNING: not removing hydrogen atom without neighbors


In [ ]:
PREPROCESSED_FILE = "/home/michpir/Documents/PROJECTS/dataset/train_homo_lumo_graph_pow.output"
with open(PREPROCESSED_FILE, 'wb') as fid:
  pickle.dump(data_list_train, fid)
  fid.close() 

In [7]:


def process_and_save_chunk(chunk, file_name):
    # Convert the chunk of data to graph representations
    data_list = gp.create_graph_data(chunk, np.Inf, np.Inf, np.Inf, input_source="homo-lumo")
    
    # Append the processed chunk to the file
    with open(file_name, 'ab') as fid:
        pickle.dump(data_list, fid)

# Define the size of each chunk
chunk_size = 50000  # Adjust this based on your memory capacity

# File to save processed data
PREPROCESSED_FILE = "/home/michpir/Documents/PROJECTS/dataset/test_homo_lumo_graph_pow.output"

# Ensure the output file is empty initially
open(PREPROCESSED_FILE, 'wb').close()

# Process the dataset in chunks
for start_idx in range(0, len(testchallenge_idx), chunk_size):
    end_idx = min(start_idx + chunk_size, len(testchallenge_idx))
    train_chunk = [dataset[idx] for idx in train_idx[start_idx:end_idx]]
    process_and_save_chunk(train_chunk, PREPROCESSED_FILE)

    print(f"Processed chunk {start_idx}-{end_idx}")


Processed chunk 0-50000
Processed chunk 50000-100000
Processed chunk 100000-147432


In [11]:
print("hello")

hello
